In [2]:
import numpy as np
import scipy
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ParameterVector
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp, Statevector, state_fidelity, Operator
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.primitives import Estimator
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.linalg import expm, cosm
import warnings

In [4]:
def get_hamiltonian(nqubits):
    J=1/np.sqrt(2)
    ZZ_tuples = [('ZZ', [i, i+1], J) for i in range(nqubits-1)]
    ZZ_tuples += [('ZZ', [nqubits-1, 0], J)]
    X_tuples = [("X", [i], J) for i in range(nqubits)]
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *X_tuples], num_qubits=nqubits)
    return hamiltonian.simplify()
    
def ansatz_hamiltonian(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for i in range(nqubits):
        circuit.h(i)
    for idepth in range(depth):
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0
            circuit.rzz(params[t], i, j)
            t += 1
        circuit.barrier()
        for i in range(nqubits):
            if i < (nqubits - 1):
                j = i + 1
            else:
                j = 0    
            circuit.rx(params[t], i)
            t += 1
    return circuit    
    
def ansatz_hea(nqubits, depth, params):
    circuit = QuantumCircuit(nqubits)
    t = 0
    for idepth in range(depth):
        for i in range(nqubits):
            circuit.ry(params[t], i)
            t += 1
        circuit.barrier()
        for i in range(nqubits-1):
            circuit.cx(i, i+1)
    return circuit    
   
def cost_function1(params, nqubits, depth, error_rate):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hamiltonian(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,'cx')
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real
    
def cost_function2(nqubits, error_rate, params):
    hamiltonian =  get_hamiltonian(nqubits)
    circuit =  ansatz_hea(nqubits, depth,params)
    circuit = circuit.decompose()
    noise_model = NoiseModel()
    error = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])
    error1 = depolarizing_error(error_rate*10, 2)
    noise_model.add_all_qubit_quantum_error(error1,['cx'])
    sim_d = AerSimulator(noise_model=noise_model)
    circuit.save_statevector() 
    if error_rate == 0:
        simulator = AerSimulator()
    else:
        simulator = sim_d
        circuit = transpile(circuit, sim_d)
        #noise_result = sim_d.run(circ_noise, shots=1).result()
    result = simulator.run(circuit).result()
    u=result.data(0)['statevector'].data
    expectation = (u.conj().dot(hamiltonian.to_matrix())).dot(u)
    return expectation.real

def callback(intermediate_result):
    print(
        f"{intermediate_result.fun}"
        )
    #with open('intermediate_values_hva.txt', 'a') as file:
     #   file.write(f'Intermediate values: {intermediate_result.fun}\n')
    


In [5]:
np.random.seed(42)  # Example seed
nqubits = 4
J = 1/np.sqrt(2)
depth = 2
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)

-1.1465461343347791
-2.602898247291077
-2.848094012158368
-3.206002304191979
-3.319994326750565
-3.4384044683031587
-3.53736241079297
-3.6054730572544242
-3.6484027579854708
-3.662023065487695
-3.6704427744721184
-3.6765936854301247
-3.6807427635642345
-3.6865485932013033
-3.6930312588906897
-3.694791979762086
-3.6951774979858847
-3.6953272773565558
-3.6954448664900577
-3.6954921883044745
-3.695513649637448
-3.695517967045898
-3.6955181144876983
-3.695518129745835
-3.6955181300339923


In [9]:
np.random.seed(42)  # Example seed
nqubits = 8
J = 1/np.sqrt(2)
depth = 3
error_rate = 0
parameter = np.array(np.random.random(2*nqubits*depth))
#ansatz_hamiltonian(nqubits, depth, parameter).draw('mpl')
estimate_val_hva = minimize(cost_function1, parameter, args=(nqubits, depth, error_rate), method="BFGS", tol=1e-5, options={'disp': False}, callback=callback)#, args=(nqubits, error_rate)

-0.9905692515755558
-3.6197504290283975
-5.200674462326649
-5.973694170876433
-6.011125328238565
-6.347514519410105
-6.756220529313845
-6.9150283700816875
-6.93209174945375
-6.945294317568429
-6.958535528042241
-6.966055393102237
-6.970821826033266
-6.97531837172391
-6.9775770916858875
-6.980717970380259
-6.983519372032209
-6.986681305105318
-6.990423603041158
-6.9954091900981705
-6.999147885046877
-7.003520467299889
-7.006908773007503
-7.009629013600507
-7.011544702427889
-7.01366159904866
-7.014932747393546
-7.015467683365535
-7.016038988299014
-7.0164419375691285
-7.016843042784033
-7.017424253421718
-7.017925930832738
-7.018493377536426
-7.019231900294368
-7.01999845541088
-7.020849673780115
-7.0219459083253355
-7.022238022148327
-7.022687161847659
-7.023391966763172
-7.024295235604455
-7.02491681760316
-7.025614461390854
-7.026706022912716
-7.027682511193494
-7.028522788024732
-7.029924586544922
-7.03194503714421
-7.033451931164286
-7.0355117385197135
-7.038346622332135
-7.0396492

KeyboardInterrupt: 

In [74]:
hamiltonian = get_hamiltonian(4)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-3.695518130045147


In [71]:
hamiltonian = get_hamiltonian(8)
H_array = hamiltonian.to_matrix()
eval, _ = np.linalg.eigh(H_array)
# print(eval)
emin = eval[0]
emax = eval[-1]
print(emin)

-7.249019570823107


TO BE DONE